<a href="https://colab.research.google.com/github/thongle28625/ttnt/blob/main/BAOCAO2_AKT%26ASTAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BÀI TOÁN AKT**

In [2]:
import copy
from heapq import heappush, heappop
# ================= CẤU HÌNH =================
# 15-puzzle (4x4)
n = 4
# 4 hướng di chuyển: xuống, trái, lên, phải
hang = [1, 0, -1, 0]
cot = [0, -1, 0, 1]
# ================= HÀNG ĐỢI ƯU TIÊN =================
class HangDoiUuTien:
    def __init__(self):
      # Heap dùng để lưu các node
        self.heap = []
        # Thêm một node vào hàng đợi
    def push(self, x):
        heappush(self.heap, x)
        # Lấy node có giá trị f(n) nhỏ nhất
    def pop(self):
        return heappop(self.heap)
        # Kiểm tra hàng đợi rỗng
    def empty(self):
        return len(self.heap) == 0
# ================= NODE =================
class Node:
    def __init__(self, cha, ma_tran, vi_tri_trong, h, g):
        self.cha = cha
        self.ma_tran = ma_tran
        self.vi_tri_trong = vi_tri_trong
        self.h = h  # heuristic
        self.g = g  # số bước đi
    # So sánh theo f(n) = g + h
    def __lt__(self, node_khac):
        return (self.h + self.g) < (node_khac.h + node_khac.g)
# ================= ĐỌC FILE =================

def doc_file(ten_file):
    ban_dau = []
    dich = []
    # Đọc file và loại bỏ dòng trống
    with open(ten_file, "r", encoding="utf-8") as f:
        dong = [l.strip() for l in f.readlines() if l.strip()]
        # Chuyển dòng thành danh sách số
    ban_dau_flat = list(map(int, dong[0].split()))
    dich_flat = list(map(int, dong[1].split()))
    # Chuyển danh sách 1 chiều thành ma trận 4x4
    for i in range(n):
        ban_dau.append(ban_dau_flat[i*n : (i+1)*n])
        dich.append(dich_flat[i*n : (i+1)*n])
    return ban_dau, dich
# ================= HEURISTIC =================
def tinh_heuristic(ma_tran, dich):
    dem = 0
    for i in range(n):
        for j in range(n):
          # Không tính ô trống (0)
            if ma_tran[i][j] != 0 and ma_tran[i][j] != dich[i][j]:
                dem += 1
    return dem
# ================= TẠO NODE CON =================
def tao_node_con(ma_tran, vi_tri_trong, vi_tri_moi, g, cha, dich):
   # Sao chép ma trận để không làm thay đổi node cha
    ma_tran_moi = copy.deepcopy(ma_tran)
    x1, y1 = vi_tri_trong
    x2, y2 = vi_tri_moi
    # Hoán đổi ô trống với ô kề bên
    ma_tran_moi[x1][y1], ma_tran_moi[x2][y2] = ma_tran_moi[x2][y2], ma_tran_moi[x1][y1]
    # Tính heuristic cho trạng thái mới
    h = tinh_heuristic(ma_tran_moi, dich)
    return Node(cha, ma_tran_moi, vi_tri_moi, h, g)
# ================= IN MA TRẬN =================
def in_ma_tran(ma_tran):
    for dong in ma_tran:
        print(*dong)
    print()
# ================= KIỂM TRA BIÊN =================
def hop_le(x, y):
    return 0 <= x < n and 0 <= y < n
# ================= IN ĐƯỜNG ĐI =================
def in_duong_di(node):
    if node is None:
        return
    in_duong_di(node.cha)
    in_ma_tran(node.ma_tran)
# ================= A* GIẢI 15-PUZZLE =================
def giai_puzzle(ban_dau, dich):
    hang_doi = HangDoiUuTien()
    da_tham = set() # Lưu các trạng thái đã duyệt
    # Tìm vị trí ô trống trong trạng thái ban đầu
    for i in range(n):
        for j in range(n):
            if ban_dau[i][j] == 0:
                vi_tri_trong = [i, j]
    # Tạo node gốc
    goc = Node(None, ban_dau, vi_tri_trong,
               tinh_heuristic(ban_dau, dich), 0)
    hang_doi.push(goc)

    # Vòng lặp chính của A*
    while not hang_doi.empty():
        hien_tai = hang_doi.pop()
        # Chuyển trạng thái thành tuple để lưu vào set
        trang_thai = tuple(map(tuple, hien_tai.ma_tran))
        if trang_thai in da_tham:
            continue
        da_tham.add(trang_thai)
         # Nếu heuristic = 0 → đạt trạng thái đích
        if hien_tai.h == 0:
          print(" ĐÃ TÌM THẤY LỜI GIẢI")
          print(" Số bước cần di chuyển:", hien_tai.g)
          print("\n CÁC TRẠNG THÁI:")
          in_duong_di(hien_tai)
          return
        # Sinh các node con
        for i in range(4):
            x_moi = hien_tai.vi_tri_trong[0] + hang[i]
            y_moi = hien_tai.vi_tri_trong[1] + cot[i]

            if hop_le(x_moi, y_moi):
                con = tao_node_con(
                    hien_tai.ma_tran,
                    hien_tai.vi_tri_trong,
                    [x_moi, y_moi],
                    hien_tai.g + 1,
                    hien_tai,
                    dich
                )
                hang_doi.push(con)
# ================= MAIN =================
if __name__ == "__main__":
  ban_dau, dich = doc_file("/content/data.txt")
  print(" TRẠNG THÁI BAN ĐẦU:")
  in_ma_tran(ban_dau)
  print(" TRẠNG THÁI ĐÍCH:")
  in_ma_tran(dich)
  giai_puzzle(ban_dau, dich)


#1 2 3 4 5 6 7 8 9 10 11 0 13 14 15 12
#1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0





 TRẠNG THÁI BAN ĐẦU:
1 2 3 4
5 6 7 8
9 10 11 0
13 14 15 12

 TRẠNG THÁI ĐÍCH:
1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 0

 ĐÃ TÌM THẤY LỜI GIẢI
 Số bước cần di chuyển: 1

 CÁC TRẠNG THÁI:
1 2 3 4
5 6 7 8
9 10 11 0
13 14 15 12

1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 0



**BÀI TOÁN ASTAR**

In [11]:

# ================= LỚP ĐỒ THỊ =================
class DoThi:
    def __init__(self):
        # Danh sách kề: {đỉnh: [(đỉnh_kề, trọng_số), ...]}
        self.danh_sach_ke = {}

        # Tập hợp tất cả các đỉnh
        self.tap_dinh = set()

    def doc_file(self, ten_file):
        """Đọc đồ thị từ file"""
        with open(ten_file, "r", encoding="utf-8") as f:
            for dong in f:
                dong = dong.strip()

                # Bỏ qua dòng trống hoặc comment
                if not dong or dong.startswith("#"):
                    continue

                dinh_u, dinh_v, trong_so = dong.split()
                trong_so = int(trong_so)

                self.tap_dinh.add(dinh_u)
                self.tap_dinh.add(dinh_v)

                if dinh_u not in self.danh_sach_ke:
                    self.danh_sach_ke[dinh_u] = []

                self.danh_sach_ke[dinh_u].append((dinh_v, trong_so))

    def lay_dinh_ke(self, dinh):
        """Lấy các đỉnh kề của một đỉnh"""
        return self.danh_sach_ke.get(dinh, [])

    def hien_thi_do_thi(self, dinh_bat_dau):
        """Hiển thị sơ đồ đồ thị ban đầu"""
        print("SƠ ĐỒ ĐỒ THỊ BAN ĐẦU")
        print(f"Đỉnh bắt đầu được chọn: {dinh_bat_dau}")
        for u in self.danh_sach_ke:
            for v, w in self.danh_sach_ke[u]:
                print(f"{u} ──{w}──▶ {v}")
        print()
# ================= LỚP A* =================
class GiaiAStar:
    def __init__(self, do_thi):
        self.do_thi = do_thi

    def ham_heuristic(self, dinh):
        """
        Hàm heuristic h(n)
        Ở đây chọn h(n) = 0
        => A* trở thành Dijkstra
        """
        return 0

    def tim_duong_di(self, dinh_bat_dau, dinh_dich):
        """Thuật toán A* tìm đường đi"""

        tap_mo = {dinh_bat_dau}      # OPEN
        tap_dong = set()             # CLOSED

        chi_phi_g = {dinh_bat_dau: 0}
        cha = {dinh_bat_dau: dinh_bat_dau}

        while tap_mo:
            # Chọn đỉnh có f = g + h nhỏ nhất
            dinh_hien_tai = min(
                tap_mo,
                key=lambda x: chi_phi_g[x] + self.ham_heuristic(x)
            )

            # Nếu đến đích
            if dinh_hien_tai == dinh_dich:
                duong_di = self.truy_vet_duong_di(cha, dinh_bat_dau, dinh_dich)
                return duong_di, chi_phi_g[dinh_dich]

            tap_mo.remove(dinh_hien_tai)
            tap_dong.add(dinh_hien_tai)

            for dinh_ke, trong_so in self.do_thi.lay_dinh_ke(dinh_hien_tai):
                if dinh_ke in tap_dong:
                    continue
                chi_phi_tam = chi_phi_g[dinh_hien_tai] + trong_so
                if dinh_ke not in tap_mo or chi_phi_tam < chi_phi_g.get(dinh_ke, float("inf")):
                    cha[dinh_ke] = dinh_hien_tai
                    chi_phi_g[dinh_ke] = chi_phi_tam
                    tap_mo.add(dinh_ke)

        return None, None

    def truy_vet_duong_di(self, cha, dinh_bat_dau, dinh_dich):
        """Truy vết đường đi từ đích về đầu"""
        duong_di = []
        dinh = dinh_dich

        while dinh != dinh_bat_dau:
            duong_di.append(dinh)
            dinh = cha[dinh]

        duong_di.append(dinh_bat_dau)
        duong_di.reverse()
        return duong_di


# ================= LỚP GIAO DIỆN =================
class GiaoDien:
    def __init__(self, do_thi, giai_thuat):
        self.do_thi = do_thi
        self.giai_thuat = giai_thuat

    def chon_dinh(self, thong_bao):
        """Cho phép người dùng chọn đỉnh hợp lệ"""
        while True:
            dinh = input(thong_bao).strip()
            if dinh in self.do_thi.tap_dinh:
                return dinh
            print("Đỉnh không tồn tại!")
            print("Các đỉnh hợp lệ:", sorted(self.do_thi.tap_dinh))

    def hien_thi_ket_qua(self, duong_di, tong_chi_phi):
        """Hiển thị kết quả tìm đường"""
        print("ĐÃ TÌM THẤY LỜI GIẢI")
        print("Đường đi:", duong_di)
        print("Số bước:", len(duong_di) - 1)
        print("Tổng chi phí:", tong_chi_phi)

        print("\nSƠ ĐỒ ĐƯỜNG ĐI:")
        so_do = "START: "
        for i in range(len(duong_di) - 1):
            u, v = duong_di[i], duong_di[i + 1]
            for x, w in self.do_thi.lay_dinh_ke(u):
                if x == v:
                    so_do += f"{u} ──{w}──▶ "
                    break
        so_do += f"{duong_di[-1]} GOAL"
        print(so_do)

    def chay(self):
        """Hàm chạy chính"""
        print(" DANH SÁCH ĐỈNH TRONG ĐỒ THỊ:")
        print(sorted(self.do_thi.tap_dinh))
        print()

        dinh_bat_dau = self.chon_dinh("Nhập đỉnh bắt đầu: ")
        dinh_dich = self.chon_dinh("Nhập đỉnh đích: ")
        print()

        self.do_thi.hien_thi_do_thi(dinh_bat_dau)

        duong_di, tong_chi_phi = self.giai_thuat.tim_duong_di(dinh_bat_dau, dinh_dich)

        if duong_di:
            self.hien_thi_ket_qua(duong_di, tong_chi_phi)
        else:
            print(" Không tồn tại đường đi")


# ================= MAIN =================
if __name__ == "__main__":
    do_thi = DoThi()
    do_thi.doc_file("/content/graph.txt")

    giai_astar = GiaiAStar(do_thi)
    giao_dien = GiaoDien(do_thi, giai_astar)

    giao_dien.chay()



# u v w  (đỉnh nguồn, đỉnh đích, trọng số)
#A B 1
#A C 3
#B D 2
#C D 1
#D E 4
#C E 6




 DANH SÁCH ĐỈNH TRONG ĐỒ THỊ:
['A', 'B', 'C', 'D', 'E']

Nhập đỉnh bắt đầu: A
Nhập đỉnh đích: E

SƠ ĐỒ ĐỒ THỊ BAN ĐẦU
Đỉnh bắt đầu được chọn: A
A ──1──▶ B
A ──3──▶ C
B ──2──▶ D
C ──1──▶ D
C ──6──▶ E
D ──4──▶ E

ĐÃ TÌM THẤY LỜI GIẢI
Đường đi: ['A', 'B', 'D', 'E']
Số bước: 3
Tổng chi phí: 7

SƠ ĐỒ ĐƯỜNG ĐI:
START: A ──1──▶ B ──2──▶ D ──4──▶ E GOAL
